In [1]:
#Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
#pathway to dataframes
path = r'C:\Users\ajkap\Instacart Basket Analysis'

In [3]:
#Importing customer csv file
customer = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'), index_col = False)

In [4]:
#Importing df_ords_prods_merged_aggregated
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged_aggregated_.pkl'))

# Wrangling Data for customer csv

In [8]:
customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [9]:
customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


In [10]:
customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


## Insights
-Surnam needs to be switched to 'last_name'
-State needs to be switched to 'state'
Gender change to 'gender'
-First_name needs to be switched to 'first_name'
-Age needs to be switched to 'age'
-user_id can be used to merge with ords_prods_merged dataframe
-can drop n_dependents since it is not necessary for analysis

In [11]:
#Rename columns
customer = customer.rename(columns={'First Name':'first_name', 'Surnam':'last_name', 'STATE':'state', 'Gender':'gender', 'Age':'age'})

In [9]:
#check updates
customer.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 5. Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement; for example, check for and address missing values and duplicates, and convert any mixed-type data.

In [12]:
#Check for Mixed data types
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

first_name


## The column first_name contains mixed data type

In [13]:
#Fix mixed data type by converting to a str
customer['first_name'] = customer['first_name'].astype('str')

In [14]:
#rerun to check mixed data
for col in customer.columns.tolist():
  weird = (customer[[col]].applymap(type) != customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer[weird]) > 0:
    print (col)

## Check for Missing Values

In [15]:
#Check customer csv for missing data
customer.isnull().sum()

user_id         0
first_name      0
last_name       0
gender          0
state           0
age             0
date_joined     0
n_dependants    0
fam_status      0
income          0
dtype: int64

In [16]:
customer[customer.duplicated()]

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income


## No missing values for any columns. It is good to go

## Check if there's duplicates

In [15]:
# Check for duplicates 
df_dups = customer[customer.duplicated()]

In [16]:
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income


## no duplicates found

# Merging customer with ords_prods_merged_aggregated


In [17]:
#check import
ords_prods_merged.head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,price_range_loc,busiest_day,Busiest days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_prior_orders,order_frequency_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Mid-range product,Regularly busy,Regularly busy days,Average_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Mid-range product,Regularly busy,Least busy days,Average_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Mid-range product,Regularly busy,Least busy days,Most_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Mid-range product,Least busy,Least busy days,Average_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Mid-range product,Least busy,Least busy days,Most_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,Mid-range product,Regularly busy,Regularly busy days,Average_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
6,550135,1,prior,7,1,9,20.0,196,1,1,...,Mid-range product,Regularly busy,Busiest days,Most_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,Mid-range product,Regularly busy,Busiest days,Most_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,Mid-range product,Regularly busy,Busiest days,Most_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,Mid-range product,Least busy,Least busy days,Average_orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [18]:
#Recheck data types of orders/products data set
ords_prods_merged.dtypes

order_id                     int64
user_id                      int64
eval_set                    object
order_number                 int64
order_dow                    int64
order_hour_of_day            int64
days_since_prior_order     float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
Unnamed: 0_y                 int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_day                 object
Busiest days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
average_price              float64
spending_flag               object
median_prior_orders        float64
order_frequency_flag        object
dtype: object

In [19]:
ords_prods_merged.shape

(32406041, 26)

In [20]:
customer.shape

(206209, 10)

In [6]:
ords_prods_merged['prices'].max()

25.0

In [ ]:
ords_prods_merged.loc[ords_prods_merged['prices'] >100, 'prices'] = np.nan

In [17]:
#dropping merge column
ords_prods_merged = ords_prods_merged.drop(columns = ['_merge'])

In [18]:
#merge data sets on user_id
df_merged_clean = customer.merge(ords_prods_merged, on = 'user_id', indicator = True)

In [22]:
#checking new merged dataframe
df_merged_clean.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,fam_status,income,...,busiest_day,Busiest days,busiest_period_of_day,max_order,loyalty_flag,average_price,spending_flag,median_prior_orders,order_frequency_flag,_merge
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most_orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy days,Most_orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Busiest days,Most_orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Regularly busy days,Most_orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,Regularly busy,Least busy days,Most_orders,8,New customer,7.988889,Low spender,19.0,Regular customer,both


In [23]:
#checking shape of new dataframe
df_merged_clean.shape

(32406041, 35)

In [25]:
#Export to pickle file
df_merged_clean.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_customer_merge_.pkl'))